In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from gensim.corpora.dictionary import Dictionary

import pandas as pd
import re
from tqdm import tqdm
from nltk.tokenize import word_tokenize

from gensim.models import LdaModel

from nltk.corpus import stopwords

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

import numpy as np

from deepxml.dataset import MultiLabelDataset
from deepxml.models import Model

from pathlib import Path

from deepxml.evaluation import get_p_1, get_p_5, get_p_10, get_n_1, get_n_5, get_n_10

from deepxml.cornet import CorNet

In [6]:
df = pd.read_parquet("../data/habr_posts_dataset.parquet")

In [7]:
df

,post_id,author,title,tags,text
0,807711,Kaspersky_Lab,Security Week 2416: уязвимость в серверных мат...,"[Блог компании «Лаборатория Касперского», Инфо...",На прошлой неделе исследователи компании Binar...
1,807709,markshevchenko,Вычислительные выражения: Подробнее про типы-о...,"[.NET, Функциональное программирование, F#]",В предыдущем посте мы познакомились с концепци...
2,807707,ru_vds,Угадай местоположение льдины с арктическим ЦОД...,"[Блог компании RUVDS.com, Хостинг, Системное а...","Как вы наверняка знаете, 12 апреля RUVDS успеш..."
3,807705,shaddyk,Запустили проект с НСИС по повышению качества ...,"[Блог компании HFLabs, Открытые данные, IT-ком...",НСИС — оператор единой автоматизированной инфо...
4,807703,VokaMut,Тестируем AI на создании прикладного приложения,"[Веб-разработка, Искусственный интеллект, Natu...","Всем привет, я Григорий Тумаков, CTO в Моризо ..."
...,...,...,...,...,...
4371,797723,Squirrelfm,Анатомия эффективного собеседования. Что делат...,"[Блог компании Raft, Управление персоналом, Ка...","Я провел много собеседований за свою карьеру, ..."
4372,797721,maybe_elf,None,"[Законодательство в IT, Искусственный интеллек...",В OpenAI рассказали о мотивах Илона Маска при ...
4373,797719,ar4w,SD-Access без DNAC и ISE,"[Информационная безопасность, IT-инфраструктур...",В 2019 мы закупили комплект оборудования и лиц...
4374,797715,maybe_elf,Meta* удалит все учётные записи Oculus в конце...,"[Управление сообществом, Разработка под AR и V...",Meta* в электронной рассылке сообщила пользова...


In [8]:
def tokenize(sentence: str, sep='/SEP/'):
    # We added a /SEP/ symbol between titles and descriptions such as Amazon datasets.
    return [token.lower() if token != sep else token for token in word_tokenize(sentence)
            if len(re.sub(r'[^\w]', '', token)) > 0]

In [9]:
stop_words = stopwords.words('russian')

In [10]:
tokenized = []
for post in tqdm(df["text"]):
    tokenized.append(tokenize(post))

  0%|          | 0/4376 [00:00<?, ?it/s]

100%|██████████| 4376/4376 [00:24<00:00, 178.95it/s]


In [11]:
tokenized = [[i for i in doc if i not in stop_words] for doc in tokenized]

In [12]:
tokenized = [[i for i in doc if not i.isdigit()] for doc in tokenized]

In [13]:
mlb = MultiLabelBinarizer(sparse_output=True)

In [14]:
labels = mlb.fit_transform(df["tags"].to_list())

In [15]:
train_texts, val_text, train_labels, val_labels = train_test_split(tokenized, labels, test_size=0.1)

In [16]:
d = Dictionary(train_texts)

In [17]:
corpus = [d.doc2bow(text) for text in train_texts]

In [18]:
lda = LdaModel(corpus, num_topics=train_labels.shape[1])

In [19]:
def get_lda_topics(model, num_topics):
    word_dict = {}
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20)
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [d.get(int(i[0])) for i in words]
    return pd.DataFrame(word_dict)

In [20]:
get_lda_topics(lda, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,это,медведи,quest,это,это,это,это,стекла,implicit,это
1,компилятора,column,meta,weights,словари,gpt,image,солнца,typeclass,красном
2,альфа-версии,льда,цветные,контрольных,данных,держит,которые,это,квест,солнце
3,которые,гренландии,устройство,os,например,писатель,сайта,температуре,это,death
4,также,composable,вадим,usememo,время,которые,нужно,охлаждения,букв,складов
5,время,это,это,al,которые,chatgpt,например,галактик,selectel,которые
6,например,лёд,копий,данных,который,компании,docker,звёзд,scala,время
7,системы,которые,также,usecallback,volume,рассуждать,run,млечный,мерч,секретами
8,данных,бетона,р,которые,просто,очень,несколько,телескопа,также,раствор
9,который,айсберг,весе,co_await,данные,могут,поэтому,массивных,спрятанные,который


In [21]:
lda[corpus][4]

[(567, 0.9915344)]

In [22]:
train_embs = np.zeros(shape=(len(train_texts), train_labels.shape[1]))
for i, doc in enumerate(corpus):
    for idx, val in lda[doc]:
        train_embs[i, idx] = val

In [26]:
class LDACorrectionNet(nn.Module):
    def __init__(self, num_labels, bottlenack_size):
        super().__init__()
        self.linear1 = nn.Linear(num_labels, bottlenack_size, dtype=float)
        self.act = torch.sigmoid
        self.linear2 = nn.Linear(bottlenack_size, num_labels, dtype=float)

    def forward(self, input):
        x = self.linear1(input)
        x = self.act(x)
        x = self.linear2(x)

        return x

In [34]:
class CorNetLDACorrectionNet(nn.Module):
    def __init__(self, num_labels, bottlenack_size):
        super().__init__()
        self.lda_corect_net = LDACorrectionNet(num_labels, bottlenack_size)
        self.cor_net = CorNet(num_labels)

    def forward(self, input):
        raw_logits = self.lda_corect_net(input)
        raw_logits = raw_logits.float()
        corr_logits = self.cor_net(raw_logits)

        return corr_logits

In [35]:
model = Model(network=CorNetLDACorrectionNet, 
              bottlenack_size=300, num_labels=train_labels.shape[1])

In [36]:
train_loader = DataLoader(MultiLabelDataset(train_embs, train_labels),
                          8, shuffle=True)

In [37]:
val_docs = [d.doc2bow(text) for text in val_text]

In [38]:
val_embs = np.zeros(shape=(len(val_text), train_labels.shape[1]))
for i, doc in enumerate(val_docs):
    for idx, val in lda[doc]:
        val_embs[i, idx] = val

In [39]:
val_loader = DataLoader(MultiLabelDataset(val_embs, val_labels),
                          8, shuffle=False)

In [40]:
model.train(train_loader, val_loader)

/home/daniil/programming/PapersTagsPrediction/src/deepxml/optimizers.py:97: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1578.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)


0 800 train loss: 0.0623080 valid loss: 0.0283165 P@5: 0.05114 N@5: 0.06821 early stop: 0
0 1600 train loss: 0.0287368 valid loss: 0.0282236 P@5: 0.07169 N@5: 0.09652 early stop: 0
0 2400 train loss: 0.0279387 valid loss: 0.0282496 P@5: 0.06986 N@5: 0.09825 early stop: 0
0 3200 train loss: 0.0284423 valid loss: 0.0283268 P@5: 0.07397 N@5: 0.10650 early stop: 0
1 56 train loss: 0.0287314 valid loss: 0.0285081 P@5: 0.06301 N@5: 0.09176 early stop: 0
1 856 train loss: 0.0280990 valid loss: 0.0281776 P@5: 0.06804 N@5: 0.09079 early stop: 0
1 1656 train loss: 0.0283931 valid loss: 0.0282530 P@5: 0.06575 N@5: 0.09510 early stop: 0
1 2456 train loss: 0.0278713 valid loss: 0.0283207 P@5: 0.05662 N@5: 0.07781 early stop: 0
1 3256 train loss: 0.0282697 valid loss: 0.0282807 P@5: 0.05114 N@5: 0.08279 early stop: 0
2 112 train loss: 0.0279791 valid loss: 0.0283045 P@5: 0.07169 N@5: 0.10222 early stop: 0
2 912 train loss: 0.0278194 valid loss: 0.0279128 P@5: 0.06530 N@5: 0.10215 early stop: 0
2 171

In [41]:
val_res = model.predict(val_loader)

In [42]:
metrics = [metric(val_res[1], val_labels) for metric in [get_p_1, get_p_5, get_p_10, get_n_1, get_n_5, get_n_10]]
metrics

[0.3881278538812785,
 0.21552511415525114,
 0.14132420091324202,
 0.3881278538812785,
 0.35693699381598476,
 0.4062039630858612]

In [51]:
metrics = [metric(val_res[1], val_labels) for metric in [get_p_1, get_p_5, get_p_10, get_n_1, get_n_5, get_n_10]]
metrics

[0.4269406392694064,
 0.2324200913242009,
 0.1506849315068493,
 0.4269406392694064,
 0.3950904912715572,
 0.44634322386189507]